# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

In [ ]:
features = pandas.read_pickle("features.pkl")

In [ ]:
features['sum'] = (features['playerName'] + features['college'] + features['city'] + features['state'] + features['highSchool'] + features['year'] + features['position'])/7
features.describe()

,ID,playerName,college,city,state,highSchool,position,year,sum
count,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08
mean,3.325932e-04,1.526350e-01,2.815054e-01,1.328318e-01,9.206913e-02,1.266977e-01,2.889289e-02,8.531899e-01,2.382603e-01
std,1.823411e-02,9.488154e-02,3.255962e-01,1.167102e-01,2.891235e-01,9.654791e-02,1.675055e-01,3.539165e-01,6.015086e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.428571e-01
25%,0.000000e+00,8.333333e-02,1.000000e-01,7.142857e-02,0.000000e+00,6.666667e-02,0.000000e+00,1.000000e+00,2.017625e-01
50%,0.000000e+00,1.538462e-01,1.538462e-01,1.111111e-01,0.000000e+00,1.200000e-01,0.000000e+00,1.000000e+00,2.225552e-01
75%,0.000000e+00,2.142857e-01,2.727273e-01,1.875000e-01,0.000000e+00,1.764706e-01,0.000000e+00,1.000000e+00,2.504502e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
# Fun to run but not necessary
#features.boxplot(column=['playerName', 'college', 'city', 'state', 'highSchool', 'position', 'year'])

In [ ]:
not_id = features[features['ID'] != 1]
not_id = (not_id.loc[not_id['sum'] > .945])

In [ ]:
not_id.describe()

,ID,playerName,college,city,state,highSchool,position,year,sum
count,256.0,256.000000,256.0,256.000000,256.0,256.000000,256.0,256.0,256.000000
mean,0.0,0.794609,1.0,0.999566,1.0,0.991917,1.0,1.0,0.969442
std,0.0,0.095027,0.0,0.006944,0.0,0.040516,0.0,0.0,0.013813
min,0.0,0.615385,1.0,0.888889,1.0,0.680000,1.0,1.0,0.945055
25%,0.0,0.714286,1.0,1.000000,1.0,1.000000,1.0,1.0,0.957773
50%,0.0,0.812500,1.0,1.000000,1.0,1.000000,1.0,1.0,0.971429
75%,0.0,0.866667,1.0,1.000000,1.0,1.000000,1.0,1.0,0.979932
max,0.0,0.950000,1.0,1.000000,1.0,1.000000,1.0,1.0,0.992063


In [ ]:
not_id.to_pickle("not_id.pkl")